# Spark Streaming

In [ ]:
import string
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

In [ ]:
spark = (
    SparkSession.builder 
    .master("local") 
    .appName("BIOS-823") 
    .config("spark.executor.cores", 4) 
    .getOrCreate()    
)

## Low level API

![img](https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2017/06/apache-spark-dstream-1.jpg)

Source: https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2017/06/apache-spark-dstream-1.jpg

In [ ]:
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)
ssc.checkpoint('checkpoints')

In [ ]:
lines = ssc.textFileStream('data/inputs')
counts = (lines.
          map(lambda line: line.translate(
              str.maketrans('', '', string.punctuation))).
          flatMap(lambda line: line.split()).
          map(lambda word: (word, 1)).
          reduceByKey(lambda a, b: a + b)    
)

counts.checkpoint(30)
counts.pprint()
counts.saveAsTextFiles('counts')

ssc.start()
ssc.awaitTerminationOrTimeout(60)